# Transaction Specific Note book

This notebook only implements the transaction domain self play

**Importing Libraries**

In [1]:
import numpy as np # no use until now
import random # used for random sampling of values where applicable
import math # no use untill now
import pandas as pd # to read the excel file 
import os # to carry os operations
import copy # this is used to copy the class object
import glob
#import tensorflow as tf

### Below is the Code for storing the template sentences
**What does it do ?**

The cell below reads the excel file ('templates_for_dialogue_self_play.xlsx') and extracts the template sentences to store them in a dictionary where the key is **LABEL** 

**Why do you need a dictionary ?**

It is because in the later stages of constructing the dialog, I will need to refer this dictionary to select an appropriate template sentence to convert a an *Action* Object into an spoken dialog.

**What does the function return ?**

The function returns a dictionary of templates.

Examples of keys are : inform-intent_transaction, inform-user_account, inform-user_account-partner_name etc.

For each of these keys, there is a associated list of templates collected from template collection.

In [2]:
def make_templates(sheet_name="MAKE_TRANSACTION",previous_dictionary=None) :
    if previous_dictionary == None :
        template_dictionary = dict()
    else :
        template_dictionary = previous_dictionary
    
    for file_name in glob.glob('template_folder/*.xlsx') :
        print("opening file name {}".format(file_name))
        df = pd.read_excel(file_name,sheet_name)
        for index,row in df.iterrows() :
            if not pd.isnull(row["LABEL"]) :
                if row["LABEL"] not in template_dictionary.keys() :
                    template_sentences = set()
                else :
                    template_sentences = template_dictionary[row["LABEL"]]
            
                
                if not pd.isnull(row["TEXT"]) :
                    template_sentences.add(row["TEXT"])
            
                template_dictionary[row["LABEL"]] = template_sentences
        del df
    
    # define all the dictionaries
    template_dictionary_train = dict()
    template_dictionary_val = dict()
    template_dictionary_test = dict()
    template_dictionary_test_oot = dict()
    
    final_template_dictionary = dict()
    
    for key,value in template_dictionary.items() :
        list_of_templates = list(value)
        
        if len(list_of_templates) < 3 :
            
            list_of_templates_train = list_of_templates
            list_of_templates_val = list_of_templates
            list_of_templates_test = list_of_templates
        else :
            
            list_of_templates_train = list_of_templates[0:int(len(list_of_templates)/3)]
            list_of_templates_val = list_of_templates[int(len(list_of_templates)/3):int(2*len(list_of_templates)/3)]
            
            list_of_templates_from_train = random.sample(list_of_templates,max(len(list_of_templates_train)//2,1))
            list_of_templates_from_val = random.sample(list_of_templates,max(len(list_of_templates_val)//2,1))
            
            list_of_templates_test = list()
            list_of_templates_test.extend(list_of_templates_from_train)
            list_of_templates_test.extend(list_of_templates_from_val)
            
            list_of_templates_test_oot = list_of_templates[int(2*len(list_of_templates)/3):int(len(list_of_templates))]
        
        
        
        template_dictionary_train[key] = list_of_templates_train
        template_dictionary_val[key] = list_of_templates_val
        template_dictionary_test[key] = list_of_templates_test
        template_dictionary_test_oot[key] = list_of_templates_test
    
    final_template_dictionary["train"] = template_dictionary_train
    final_template_dictionary["val"] = template_dictionary_val
    final_template_dictionary["test"] = template_dictionary_test
    final_template_dictionary["test_oot"] = template_dictionary_test_oot
    
    return final_template_dictionary
    

**DICTIONARY FOR MAKING TRANSACTION**

In [3]:
transaction_templates = make_templates(sheet_name="MAKE_TRANSACTION",previous_dictionary=None)

opening file name template_folder\Bot Generic Responses.xlsx
opening file name template_folder\EN_guerini.xlsx
opening file name template_folder\MOJO_EN.xlsx
opening file name template_folder\User Generic Responses.xlsx


## Transaction template testing

In [4]:
print(transaction_templates["train"]["initial_slot_check"])
print(transaction_templates["train"]["check_account"])
print(transaction_templates["train"]["change_partner_name_accept"])

['api_call:initial_slot_check_api,']
['api_call:account_check_api, user_account:{}']
['accept']


### Below is the Description of the Action class

**What is the Action Class ?**

Action class creates objects that intend to capture the various aspects of the conversation for e.g "What action is performed ?", "What were the values provided ?","Was there a warning that was provided ?" etc.

**Why need the Action Class, Can't you just write simple sentences like "inform date, request amount etc." ?**

While it's a good idea to write simple sentences to make a dialog, we need to remeber that there are more things happening within the dialog apart from the conversation. Like the task of keeping track of all the values that are being provided and executing some specific api based on action etc. Also not all the actions are meant to be said, some are there for the **api_call** and **end_call**. If I go for slicing each sentence to find out what to say, it becomes more complex and difficult and also we loose that ability of scaling.

In [5]:
class Action(object) :
    def __init__(self,
                 actor=None,
                 action=None,
                 slots=None,
                 values=None,
                 message=None,
                 description=None,
                 slot_concerned=None,
                 templates=None,
                 pattern_marker=None) :
        
        self.actor = actor # who performed the action
        self.action = action # what action was performed
        self.slots = slots # what slot was dealt with 
        self.values = values # what was the value with this slot
        self.message = message # Any particular message related to the action
        self.description = description # This contain the description of the action and is never intended to be shown or appear in the actual conversation
        self.templates = templates
        self.template = None
        self.dictionary_key_found = False
        self.slot_concerned = slot_concerned
        self.pattern_marker = pattern_marker
        
        self.set_templates(self.templates)
        
        
    # standard actions to perform
    def get_actor(self) :
        return self.actor
    
    def get_action(self) :
        return self.action
    
    def get_slots(self) :
        return self.slots
    
    def get_values(self) :
        return self.values
    
    def get_message(self) :
        return self.message
    
    def get_description(self) :
        return self.description
    def template_found(self) :
        return self.dictionary_key_found
    
    def get_pattern_marker(self) :
        return self.pattern_marker
    
    def set_templates(self,new_templates=None) :
        
        if new_templates :
            #print("templates changed")
            self.templates = new_templates
        
        if self.action :
            dictionary_key = self.action
        else :
            dictionary_key = str()
        
        if self.slots :
            if len(self.slots) > 0 :
                for slot in self.slots :
                    
                    if slot == "intent" :
                        
                        if self.actor == "User" :
                            
                            dictionary_key += "-" + slot + "_" + self.values["intent"]
                            
                        else :
                            
                            dictionary_key += "-" + slot
                            
                    elif slot == "domain_description" :
                        continue
                        
                    else :
                        
                        dictionary_key += "-" + slot
        
        # if the dictionary_key exists in the template dictionary then get a template other wise set template = the action message
        self.dictionary_key = dictionary_key
        if self.templates and dictionary_key in self.templates.keys() :
            self.template = random.sample(self.templates[dictionary_key],1)[0]
            self.dictionary_key_found = True
        else :
            self.template = self.message
        
    # when called , construct a dialog from the slots and give it to the user
    
    def get_dictionary_key(self) :
        
        return self.dictionary_key
    
    def get_dialog(self,with_actor=True,templates=None) :

        
        if templates :
            self.set_templates(new_templates=templates)
        
            
        # first split the template into a list of words
        words = self.template.strip().split()
        sentence = None
        
        # only go to this loop if the actor is a User or Bot , in case of API go to else statement which will just append action and message
        if self.actor == "User" or self.actor == "Bot" :
            if self.values :
                for slot,value in self.values.items() :
                    search_slot = "{" + slot + "}"
                    if search_slot in words :
                        slot_index = words.index(search_slot)
                        words.insert(slot_index,str(value))
                
                        words.pop(slot_index+1)
                    sentence = " ".join(words)
                
            # if it's a request action then get a template requesting the slots
            elif self.action == "request" :
                
                sentence = self.template
            
            # if it's a end_call then show the action and the message given
            elif self.action == "end_call" :
                
                sentence = self.message
            
            else :
                sentence = self.message
        
        else:
            sentence = self.message
        if with_actor :
            return self.actor + " : " + sentence
        else :
            return sentence


## Loading the knowledge base

The knowledge base is a defined set of user profiles that will be used for the experiment.

In [6]:
knowledge_base = pd.read_excel('user_values.xlsx','UserValues')
list_of_user_profiles = list()

for index,row in knowledge_base.iterrows() :
    list_of_user_profiles.append(row)

In [7]:
data_frame = pd.read_excel('Game_of_Dialogs.xlsx','VARIABLE_VALUES')
user_values = dict()

for index,row in data_frame.iterrows() :
    for column in data_frame.columns :
        list_of_values = list()
        if column in user_values.keys() :
            list_of_values = user_values[column]
        if not pd.isnull(row[column]) :
            list_of_values.append(row[column])
        user_values[column] = list_of_values

### The User Class

**Who is the User Class ?**

The _User_ class our custom user who in the right sense represents our _dummy customer_ who will come to the bank for a transaction

**How is he working ?**

The _User_ Class adopts a certain _"personality"_ (a set of values) when instantiated, whenever the bot performs a specific action, it looks for the kind of request and respondes using it's _speak_ function

**Assumptions**

1. It is possible that the user has multiple accounts and different transfer limits and balances on each of them, we assume that each account has the same set of transfer limits and balances

**Creating the Transaction User**

This class creates a User that converses - communicates through actions - in the transaction domain.

In [8]:
class Transaction_user() :
    def __init__(self,
                 templates=None,
                 list_of_user_profiles=None,
                 user_values=None,
                 turn_compression=False,
                 new_api=False,
                 another_slot=False,
                 audit_more=False) :
        
        # Below is the available pool of values from which we will create a Custom user for the transaction
        self.user_names = user_values["partner_names"]
        self.user_accounts = user_values["user_accounts"]
        self.transfer_amt = user_values["amount_values"]
        self.slots = ["user_account","partner_name","amount"]
        
        self.slots_to_give = copy.deepcopy(self.slots)
        
        self.templates = templates
        self.priority_states = list()
        self.priority_actions = dict()
        self.turn_compression = turn_compression
        self.new_api = new_api
        self.another_slot = another_slot
        self.audit_more = audit_more
        
        
        self.override = False
        
        self.state_track = dict()
        self.state_track["CHANGE_PARTNER_NAME"] = 0
        self.state_track["CHANGE_AMOUNT"] = 0
        self.state_track["CHANGE_ACCOUNT"] = 0
        # create the custom user
        self.user = dict()
        
        row_chosen = random.randint(0,len(list_of_user_profiles)-1)
        self.user_chosen = list_of_user_profiles[row_chosen]
        
        self.create_user_profile(self.user_chosen)
    
    def sort_my_slots(self,slots_given) :
        
        
        if slots_given :
            
            slots_sorted = list()
            
            if "user_account" in slots_given :
                slots_sorted.append("user_account")
                slots_given.remove("user_account")
            
            if "partner_name" in slots_given :
                slots_sorted.append("partner_name")
                slots_given.remove("partner_name")
            
            if "amount" in slots_given :
                slots_sorted.append("amount")
                slots_given.remove("amount")
        
            for slot in slots_given :
                slots_sorted.append(slot)
        else :
            slots_sorted = list()
        
        return slots_sorted
    
    def create_user_profile(self,user_chosen) :
        
        # Every value is assigned randomly 
        
        # selectinng name of sender and reciever
        
        #names = random.sample(self.user_names,2)
        
        #self.user["name"] = names[0]
        self.user["name"] = user_chosen["name"]
        
        #self.user["partner_name"] = names[1]
    
        #number_of_partner_names = random.randint(1,len(self.user_names))
        #self.user["partner_names"] = random.sample(self.user_names,number_of_partner_names)
        #self.user["partner_names"].sort()
        
        list_of_partner_names = user_chosen["partner_names"].strip().split(',')
        self.user["partner_names"] = list_of_partner_names
        self.user["partner_names"].sort()
        
        self.user["partner_name"] = random.sample(self.user_names,1)[0]
        
        #selecting the usr_account to make the transaction from
        #self.user["user_account"] = random.sample(self.user_accounts,1)[0]
        
        #number_of_user_accounts = random.randint(1,len(self.user_accounts))
        #self.user["user_accounts"] = random.sample(self.user_accounts,number_of_user_accounts)
        #self.user["user_accounts"].sort()
        
        list_of_user_accounts = user_chosen["user_accounts"].strip().split(',')
        self.user["user_accounts"] = list_of_user_accounts
        self.user["user_accounts"].sort()
        
        self.user["user_account"] = random.sample(self.user_accounts,1)[0]
        
        
        # selecting the amount to be transfered
        self.user["amount"] = int(random.sample(self.transfer_amt,1)[0])
        #self.user["amount"] = int(user_chosen["amount"])
        
        
        # selecting the balance of the user
        #self.user["balance"] = random.sample(self.user_balances,1)[0]
        self.user["balance"] = int(user_chosen["balance"])
        
        # selecting the limit of the user
        #self.user["limit"] = random.sample(self.transaction_limit,1)[0]
        self.user["limit"] = int(user_chosen["limit"])
        
        
        # setting up the max_transferable amount
        self.user["max_transferable_amt"] = min(self.user["limit"],self.user["balance"])
        
        # setting up the intent
        self.user["intent"] = "transaction"
        self.user["domain_description"] = "transaction_memory_network"
    
    # Returns the respective value of the slot
    def get_value(self,slot_asked) :
        
        return self.user[slot_asked]
    
    def remove_slot(self,slot_given) :
        if slot_given in self.slots :
            self.slots.remove(slot_given)
        
    
    def perform_random_action(self,bot_action) :
        
        values_to_give = dict()
        actual_actor = None
        actual_action = None
        slot_to_give = list()
        pattern_to_give = list()
        
        if bot_action.get_description() == "API_CALL" :
            
            actual_actor = "API"
            actual_action = None
            
            accept_key = "transaction_success"
            accept_message = self.templates[accept_key][0]
            
            reject_key = "transaction_failed"
            reject_message = self.templates[reject_key][0]
            
            #accept_message = "api_response:api_call, api_result:success"
            #reject_message = "api_response:api_call, api_result:failed"
            slot_concerned = "api"
        
        elif bot_action.get_description() == "CHANGE_ACCOUNT" :
            
            
            slot_to_give.append("user_account")
            new_account = random.sample(self.user_accounts,1)[0]
            while new_account == self.user["user_account"] :
                new_account = random.sample(self.user_accounts,1)[0]
                
            self.user["user_account"] = new_account
            
            if self.state_track["CHANGE_ACCOUNT"] > 2 :
                self.override = True
                new_account = random.sample(self.user["user_accounts"],1)[0]
                self.user["user_account"] = new_account
                
            values_to_give["user_account"] = new_account
            
            actual_actor = "User"
            actual_action = None
            
            accept_key = "change_account_accept"
            accept_message = self.templates[accept_key][0]
            
            reject_key = "change_account_reject"
            reject_message = self.templates[reject_key][0]
            
            #accept_message = "accept"
            #reject_message = "reject"
            slot_concerned = "user_account"
            
            if self.audit_more :
                slots_to_change = copy.deepcopy(self.slots_to_give)
                slots_to_change.remove("user_account")
                
                slot_chosen_to_change = random.sample(slots_to_change,1)[0]
                
                slot_choice_list = list()
                
                if slot_chosen_to_change == "partner_name" :
                    slot_choice_list = self.user_names
                elif slot_chosen_to_change == "amount" :
                    slot_choice_list = self.transfer_amt
                    
                new_value = random.sample(slot_choice_list,1)[0]
                
                while new_value == self.user[slot_chosen_to_change] :
                    new_value = random.sample(slot_choice_list,1)[0]
                    
                values_to_give[slot_chosen_to_change] = new_value
                
                slot_to_give.append(slot_chosen_to_change)
                pattern_to_give.append("audit_more")
                
                
                accept_key = "change_account_accept_audit_more"
                accept_message = self.templates[accept_key][0]
                accept_message = accept_message.format(new_account,slot_chosen_to_change,new_value)
                #accept_message = "accept use {} and change {} to {}".format(new_account,slot_chosen_to_change,new_value)
                
            
                
            if self.turn_compression :
                
                accept_key = "change_account_accept_turn_compression"
                accept_message = self.templates[accept_key][0]
                accept_message = accept_message.format(new_account)
                
                #accept_message = "accept use {}".format(new_account)
                pattern_to_give.append("turn_compression")
                
            self.state_track["CHANGE_ACCOUNT"] += 1
        
        elif bot_action.get_description() == "CHANGE_AMOUNT" :
            
            slot_to_give.append("amount")
            if self.state_track["CHANGE_AMOUNT"] > 2 :
                self.override = True
                
            actual_actor = "User"
            actual_action = None
            
            accept_key = "change_amount_accept"
            accept_message = self.templates[accept_key][0]
            
            reject_key = "change_amount_reject"
            reject_message = self.templates[reject_key][0]
            
            #accept_message = "accept"
            #reject_message = "reject"
            slot_concerned = "amount"
            self.user["amount"] = self.user["max_transferable_amt"]
            values_to_give["amount"] = self.user["max_transferable_amt"]
            
            self.state_track["CHANGE_AMOUNT"] += 1
        
        elif bot_action.get_description() == "CHANGE_PARTNER_NAME" :
            
            
            
            slot_to_give.append("partner_name")
                
            new_partner_name = random.sample(self.user_names,1)[0]
            
            while new_partner_name == self.user["name"] or new_partner_name == self.user["partner_name"] :
                new_partner_name = random.sample(self.user_names,1)[0]
                
            self.user["partner_name"] = new_partner_name
            
            if self.state_track["CHANGE_PARTNER_NAME"] > 2 :
                self.override = True
                new_partner_name = random.sample(self.user["partner_names"],1)[0]
                self.user["partner_name"] = new_partner_name
                
                
            values_to_give["partner_name"] = new_partner_name
            
            actual_actor = "User"
            actual_action = None
            
            accept_key = "change_partner_name_accept"
            accept_message = self.templates[accept_key][0]
            
            reject_key = "change_partner_name_reject"
            reject_message = self.templates[reject_key][0]
            
            #accept_message = "accept"
            #reject_message = "reject"
            slot_concerned = "partner_name"
            
            if self.audit_more :
                slots_to_change = copy.deepcopy(self.slots_to_give)
                slots_to_change.remove("partner_name")
                
                slot_chosen_to_change = random.sample(slots_to_change,1)[0]
                
                slot_choice_list = list()
                
                if slot_chosen_to_change == "user_account" :
                    slot_choice_list = self.user_accounts
                elif slot_chosen_to_change == "amount" :
                    slot_choice_list = self.transfer_amt
                    
                new_value = random.sample(slot_choice_list,1)[0]
                pattern_to_give.append("audit_more")
                
                while new_value == self.user[slot_chosen_to_change] :
                    new_value = random.sample(slot_choice_list,1)[0]
                    
                values_to_give[slot_chosen_to_change] = new_value
                slot_to_give.append(slot_chosen_to_change)
                
                
                accept_key = "change_partner_name_accept_audit_more"
                accept_message = self.templates[accept_key][0]
                accept_message = accept_message.format(new_partner_name,slot_chosen_to_change,new_value)
                
                #accept_message = "accept use {} and change {} to {}".format(new_partner_name,slot_chosen_to_change,new_value)

            if self.turn_compression :
                
                accept_key = "change_partner_name_accept_turn_compression"
                accept_message = self.templates[accept_key][0]
                
                #accept_message = "accept go for {}".format(new_partner_name)
                pattern_to_give.append("audit_more")
                
            self.state_track["CHANGE_PARTNER_NAME"] += 1
        
        elif bot_action.get_description() == "CONFIRM_TRANSACTION" :
            
            actual_actor = "User"
            actual_action = None
            
            accept_key = "confirm_transaction_accept"
            accept_message = self.templates[accept_key][0]
            
            reject_key = "confirm_transaction_reject" 
            reject_message = self.templates[reject_key][0]
            
            
        else :
            actual_actor = "User"
            
            accept_key = "general_accept"
            accept_message = self.templates[accept_key][0]
            #accept_message = "accept"
            
            reject_key = "general_reject"
            reject_message = self.templates[reject_key][0]
            #reject_message = "reject"
            
        toss = random.randint(0,100)
        if self.override or toss > 10 :
            self.override = False
            user_action = Action(actor=actual_actor,
                                 action=actual_action,
                                 slots=slot_to_give,
                                 values=values_to_give,
                                 message=accept_message,
                                 description="ACCEPT_REQUEST",
                                 templates=self.templates,
                                 pattern_marker=pattern_to_give)
        else :
            user_action = Action(actor=actual_actor,
                                 action=actual_action,
                                 slots=slot_to_give,
                                 values=values_to_give,
                                 message=reject_message,
                                 description="REJECT_REQUEST",
                                 templates=self.templates,
                                 pattern_marker=None)
        return user_action
    # This is the function that converses with the bot through 'Action' Objects
    def speak(self,bot_action) :
        user_action = None
        if bot_action.get_action() == "api_call" :
            
            user_action = self.api_response(bot_action)            

        elif bot_action.get_action() == "request" :
            
            if bot_action.get_slots() != None :
                
                if bot_action.get_slots()[0] != "intent" :
                    
                    slot_to_inform = bot_action.get_slots()[0]
                    
                    if self.another_slot and self.slots :
                        slots_to_choose_from = copy.deepcopy(self.slots)
                        if len(slots_to_choose_from) > 1 :
                            self.remove_slot(slot_to_inform)
                            #slots_to_choose_from.remove(slot_to_inform)
                            
                        slot_chosen_to_inform = random.sample(slots_to_choose_from,1)[0]
                        value_for_other_slot = self.get_value(slot_chosen_to_inform)
                        
                        user_action = Action(actor="User",
                                             action="inform",
                                             slots=[slot_chosen_to_inform],
                                             values={slot_chosen_to_inform : value_for_other_slot},
                                             message="Providing value for {}".format(slot_chosen_to_inform),
                                             description="ANOTHER_SLOT_VALUE",
                                             templates=self.templates,
                                             pattern_marker=["another_slot"])
                        
                        self.remove_slot(slot_chosen_to_inform)
                        #self.slots.remove(slot_chosen_to_inform)
                    else :
                        
                        user_value = self.get_value(slot_to_inform)
                    
                        user_action = Action(actor="User",
                                             action="inform",
                                             slots=bot_action.get_slots(),
                                             values={bot_action.get_slots()[0] : user_value},
                                             message="Providing value for {}".format(bot_action.get_slots()[0]),
                                             slot_concerned=bot_action.get_slots()[0],
                                             templates=self.templates)
                        
                        self.remove_slot(slot_to_inform)
                        #self.slots.remove(slot_to_inform)
                
                else :
                    rem = 0
                    pattern_to_give = list()
                    if self.new_api :
                        rem = 1
                        pattern_to_give.append("new_api")
                    number_of_slots = random.randint(0,len(self.slots))
                    
                    while number_of_slots % 2 != rem :
                        number_of_slots = random.randint(0,len(self.slots))
                        
                    slots_to_inform = random.sample(self.slots,number_of_slots)
                    
                    for slot in slots_to_inform :
                        self.remove_slot(slot)
                        
                    
                    all_slots = ["intent","domain_description"] + self.sort_my_slots(slots_to_inform)
                    
                    values_to_inform = dict()
                    
                    for slot in all_slots :
                        values_to_inform[slot] = self.user[slot]
                    
                    user_action = Action(actor="User",
                                         action="inform",
                                         slots=all_slots,
                                         values=values_to_inform,
                                         message="Providing intent",
                                         slot_concerned="intent",
                                         templates=self.templates,
                                         pattern_marker=pattern_to_give)
            else :
                user_action = self.perform_random_action(bot_action)
        
        elif bot_action.get_action() == "api_call" :
            
            user_action = self.perform_random_action(bot_action) 
        
        else :
            
            user_action = Action(actor="User",
                                 action=None,
                                 slots=None,
                                 values=None,
                                 message="<SILENCE>",
                                 templates=self.templates)
        
        return user_action
    
    # when the bot takes the role of API then, the User should assume the role of API_RESP (i.e API_RESPONSE)
    def api_response(self,bot_action) :
    
        user_action = None
        
        # if the API action asks for a account check
        if bot_action.get_description() == "API_AMOUNT_CHECK" :
            
            if self.user["amount"] > self.user["max_transferable_amt"] :
                
                key = "check_amount_failed"
                user_message = self.templates[key][0]
                user_message = user_message.format(self.user["limit"],self.user["balance"],self.user["max_transferable_amt"])
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=["limit","balance","max_transferable_amt"],
                                     values={"limit" : self.user["limit"],
                                             "balance" : self.user["balance"],
                                             "max_transferable_amt" : self.user["max_transferable_amt"]},
                                     #message="api_response:amount_check_api, limit:{},balance:{},max_transferable_amt:{}, message:'change to max_transferable_amt ?'".format(self.user["limit"],
                                      #                                                                                             self.user["balance"],
                                       #                                                                                            self.user["max_transferable_amt"]),
                                     message=user_message,
                                     slot_concerned="amount",
                                     templates=self.templates)
            
            else :
                
                key = "check_amount_success"
                user_message = self.templates[key][0]
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=["limit","balance","max_transferable_amt"],
                                     values={"limit" : self.user["limit"],
                                             "balance" : self.user["balance"],
                                             "max_transferable_amt" : self.user["max_transferable_amt"]},
                                     #message="api_response:amount_check_api, api_result:success",
                                     message=user_message,
                                     slot_concerned="amount",
                                     templates=self.templates)
        
        # if the API action askes for a initial state check
        elif bot_action.get_description() == "API_INITIAL_SLOT_CHECK" :
            
            # if the flag becomes true at the end of this segment then it means that one or more than one slots are incorrect
            flag = False
            error_message = list()
            order_of_slots = list()
            # if user account is given in the initial slots then check if it is appropriate
            if "user_account" in bot_action.get_slots() and self.user["user_account"] not in self.user["user_accounts"] :
                
                order_of_slots.append("change_account")
                
                self.priority_states.append("change_account")
                slot_message = ','.join(self.user["user_accounts"])
                
                key = "change_account"
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(slot_message)
                #bot_message = "It seems that you have not entered a valid account, you available accounts are {}, would you like change the source account ?".format(slot_message)
                self.priority_actions["change_account"] = Action(actor="Bot",
                                                                action="request",
                                                                slots=None,
                                                                values=None,
                                                                message=bot_message,
                                                                description="CHANGE_ACCOUNT",
                                                                 slot_concerned="user_account",
                                                                templates=self.templates)
            
            # if partner name is given in the initial slots then check if it is appropriate
            if "partner_name" in bot_action.get_slots() and self.user["partner_name"] not in self.user["partner_names"] :
                
                order_of_slots.append("change_partner_name")
                self.priority_states.append("change_partner_name")
                slot_message = ','.join(self.user["partner_names"])
                
                key = "change_partner_name"
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(slot_message)
                
                #bot_message = "The recipient you are trying to provide doesn't exist, available list of recipients is {}, would you like to change the recipient ?".format(slot_message)
                self.priority_actions["change_partner_name"] = Action(actor="Bot",
                                                                    action="request",
                                                                    slots=None,
                                                                    values=None,
                                                                    message=bot_message,
                                                                    description="CHANGE_PARTNER_NAME",
                                                                      slot_concerned="parnter_name",
                                                                    templates=self.templates)
            
            # if both user_account and amount are present then check if the amount satisfies the criteria
            if "user_account" in bot_action.get_slots() and "amount" in bot_action.get_slots() and self.user["amount"] > self.user["max_transferable_amt"] :
                
                order_of_slots.append("change_amount")
                self.priority_states.append("change_amount")
                
                key = "change_amount"
                
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(self.user["limit"],self.user["balance"],self.user["max_transferable_amt"])
                self.priority_actions["change_amount"] = Action(actor="Bot",
                                                                action="request",
                                                                slots=None,
                                                                values=None,
                                                                #message="It seems the amount you provided can't be processed because your transaction limit is {} and your current balance is {} so the maximum you can transfer is {}, would you like to reduce your amount to this amount ?".format(self.user["limit"],self.user["balance"],self.user["max_transferable_amt"]),
                                                                message=bot_message,
                                                                description="CHANGE_TO_MAX_TRANSFERABLE_AMT",
                                                                slot_concerned="amount",
                                                                templates=self.templates)
                
            
            # if self.priority_states is no empty then one or more than one value is incorrect then send appropriate error message
            if self.priority_states :
                order_message = ','.join(order_of_slots)
                
                key = "check_initial_slot_failed"
                user_message=self.templates[key][0]
                user_message = user_message.format(order_message)
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=self.priority_states,
                                     values=self.priority_actions,
                                     #message="api_response:initial_slot_check_api, api_result:failed, message:'{}'".format(order_message),
                                     message=user_message,
                                     slot_concerned="initial",
                                     templates=self.templates)
            
            # if everything is okay then send the correct message
            else :
                
                key = "check_initial_slot_success"
                user_message = self.templates[key][0]
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=bot_action.get_slots(),
                                     values=None,
                                     #message="api_response:initial_slot_check_api, api_result:success",
                                     message=user_message,
                                     slot_concerned="initial",
                                     templates=self.templates)
        
        # if the requested action is an account check
        elif bot_action.get_description() == "API_ACCOUNT_CHECK" :
            #print("checking account")
            if self.user["user_account"] in self.user["user_accounts"] :
                
                
                key = "check_account_success"
                user_message = self.templates[key][0]
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=["account"],
                                     values=self.user,
                                     #message="api_response:account_check_api, api_result:success",
                                     message=user_message,
                                     slot_concerned="user_account",
                                     templates=self.templates)
            
            else :
                
                slot_message = ','.join(self.user["user_accounts"])
                
                key = "check_account_failed"
                api_message = self.templates[key][0]
                api_message = api_message.format(slot_message)
                
                #api_message = "api_response:account_check_api, api_result:failed, message:'availbale list of user accounts : {}'".format(slot_message)
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=self.user["user_accounts"],
                                     values=self.user,
                                     message=api_message,
                                     slot_concerned="user_account",
                                     templates=self.templates)
        
        # if the requested action is partner name check
        elif bot_action.get_description() == "API_PARTNER_NAME_CHECK" :
            
            if self.user["partner_name"] in self.user["partner_names"] :
                
                
                key = "check_partner_name_success"
                user_message = self.templates[key][0]
                
                user_action = Action(actor="API",
                                     action=None,
                                     slots=["partner_name"],
                                     values=None,
                                     #message="api_response:partner_name_check_api, api_result:success",
                                     message=user_message,
                                     slot_concerned="partner_name",
                                     templates=self.templates)
            
            else :
                
                slot_message = ','.join(self.user["partner_names"])
                
                key = "check_partner_name_failed"
                api_message = self.templates[key][0]
                api_message = api_message.format(slot_message)
                
                #api_message = "api_response:partner_check_api, api_result:failed, message:'available list of names :{}'".format(slot_message)
                user_action = Action(actor="API",
                                     action=None,
                                     slots=self.user["partner_names"],
                                     values={"partner_names" : self.user["partner_names"]},
                                     message=api_message,
                                     slot_concerned="partner_name",
                                     templates=self.templates)
        else :
            user_action = self.perform_random_action(bot_action)
        
        return user_action            

### The Bot class

**Who is the Bot Class ?**

The Bot class can be thought to immitiate the _the system agent_ (in this case, the employee dealing in transactions).

**What is this complex piece of code below , can I understand it ?**

At first glance, No. But if it would help what it represents. The Code below is nothing more than a Finite State Machine.

**Okay !! Explain how it is working ?**

The _Bot_ or the FSM is working through a pre-determined set of states where at each state we assume that the bot will perform an action related to the field. May be the diagram below will help.
<img src="./Transaction_flow_chart.png">


Each of the circles in the above diagram is repesented by a function in the _Bot_ Class.

**What is next_state and bot_action ?**

So Basically, for each of the user's action we determine what is the appropriate *bot\_action* to be performed. This done with the help of the speak function and appropriate state function.

Once we have determined this, we determine what should be the appropriate next_state in the diagram.

If you look at the speak function it selects the appropriate function (or next_state of the diagram) from the set of states

**Creating Transaction Bot**

This bot converses and requests queries - through actions - in the transaction domain.

In [9]:
class Transaction_bot(object) :
    
    def __init__(self,
                 templates=None,
                 turn_compression=False,
                 re_order=False,
                 audit_more=False) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","partner_name","amount"]
        self.slots_to_ask = ["user_account","partner_name","amount"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "check_initial" : self.check_initial_state,
                       "user_account" : self.account_state ,
                       "check_account" : self.check_account_state,
                       "change_account" : self.change_account_state,
                       "partner_name" : self.partner_state ,
                       "check_partner" : self.check_partner_name_state,
                       "change_partner_name" : self.change_partner_name_state,
                       "amount" : self.amount_state ,
                       "check_amount" : self.check_amount_state,
                       "change_amount" : self.change_amount_state,
                       "end_call" : self.end_call_state ,
                       "balance" : self.balance_state ,
                       "confirmation_state" : self.confirmation_state,
                       "api_call" : self.api_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        self.templates = templates
        self.current_state = self.initial_state
        self.turn_compression = turn_compression
        self.re_order = re_order
        self.audit_more = audit_more
        
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "partner_name" in slots_given :
            slots_sorted.append("partner_name")
            slots_given.remove("partner_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        if user_action.get_slots() :
            for slot in user_action.get_slots() :
                
                if slot in self.slots_to_ask :
                    
                    self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        #print("next state is  == > {}".format(next_state))
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    def initial_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_slots() and "intent" in user_action.get_slots():
            
            if len(user_action.get_slots()) > 2 :
                
                next_state = "check_initial"
                key = 'initial_slot_check'
                slots_given = user_action.get_slots()[1:]
                
                slot_message = self.templates[key][0]
                
                #slot_message = "api_call:initial_slot_check,"
                for slot in slots_given :
                    if slot == "domain_description" :
                        continue
                    else :
                        slot_message += " {}:{},".format(slot,self.user_values[slot])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=user_action.get_slots(),
                                    values=None,
                                    message=slot_message[:-1],
                                    description="API_INITIAL_SLOT_CHECK",
                                    slot_concerned="initial",
                                    templates=self.templates)
            
            else :
                
                if self.re_order :
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting the values for {}".format(next_state),
                                    slot_concerned="initial",
                                    templates=self.templates)        
        else :           
            
            next_state = "initial"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["intent"],
                                values=None,
                                message="Get the intent first",
                                slot_concerned="initial",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def check_initial_state(self,user_action) :
        # if the below message is received then it means that initial check is successful and move on to the next appropriate slots
        
        if "api_result:success" in user_action.get_message() :
            
            if not self.slots_to_ask :
                
                key = 'confirm_transaction'
                
                bot_message = self.templates[key][0]
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    #message="confirm transaction ?",
                                    message=bot_message,
                                    slot_concerned="api",
                                    templates=self.templates)
            
            else :
                pattern_to_give = list()
                if self.re_order :
                    if len(self.slots_to_ask) == 1 :
                        pattern_to_give.append("re_order")
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    slot_concerned=next_state,
                                    templates=self.templates,
                                    pattern_marker=pattern_to_give)
        
        else :
            
            self.priority_states = user_action.get_slots()
            self.priority_actions = user_action.get_values()
            
            next_state = self.priority_states[0]
            bot_action = self.priority_actions[next_state]
            
            self.priority_states.remove(next_state)
        
        return next_state , bot_action
    
    def account_state(self,user_action) :
        
        # if user account has been given then 
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "user_account" in user_action.get_slots() :
            
            # remove the slot which has already been asked
            self.remove_informed_slots(user_action)
                
            # update user infomation
            user_values = user_action.get_values()
            
            # record and store all the values given by the user
            self.record_user_values(user_action)
            
            
            # pick a slot to ask randomly from the remaining slots_to_ask
            next_state = "check_account"
            
            key = "check_account"
            
            bot_message = self.templates[key][0]
            bot_message = bot_message.format(self.user_values["user_account"])
            # perform the corresponding bot information
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["user_account"],
                                values=None,
                                #message="api_call:account_check_api, user_account:{}".format(self.user_values["user_account"]),
                                message=bot_message,
                                description="API_ACCOUNT_CHECK",
                                slot_concerned="user_account",
                                templates=self.templates)
                    
        else :
            
            if user_action.get_description() == "ANOTHER_SLOT_VALUE" :
                slot_given = user_action.get_slots()[0]
                appropriate_state = self.states[slot_given]
                
                next_state, bot_action = appropriate_state(user_action)
            
            else :
                next_state = "user_account"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=["user_account"],
                                    values=None,
                                    message="requesting user to specify the account",
                                    slot_concerned="user_account",
                                    templates=self.templates)
            
        return next_state , bot_action
    
    def check_account_state(self,user_action) :
        
        self.remove_informed_slots(user_action)
        self.record_user_values(user_action)
        
        if "api_result:success" in user_action.get_message() :
            
            if "amount" in self.user_values.keys() :
                
                next_state = "check_amount"
                
                key = "check_amount"
                bot_message = self.templates[key][0]
                
                bot_message = bot_message.format(self.user_values["user_account"],self.user_values["amount"])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["limit","balance"],
                                    values=None,
                                    #message="api_call:amount_check_api, user_account:{}, amount:{}".format(self.user_values["user_account"],self.user_values["amount"]),
                                    message=bot_message,
                                    description="API_AMOUNT_CHECK",
                                    slot_concerned="amount",
                                    templates=self.templates)
            
            else :
                
                if self.priority_states :
                    next_state = self.priority_states[0]
                    bot_action = self.priority_actions[next_state]
                
                    self.priority_states.remove(next_state)
            
                elif not self.slots_to_ask :
                    
                    key = 'confirm_transaction'
                    
                    next_state = "confirmation_state"
                    
                    bot_message = self.templates[key][0]
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=None,
                                        values=None,
                                        #message="confirm transaction ?",
                                        message=bot_message,
                                        description="CONFIRM_TRANSACTION",
                                        slot_concerned="api",
                                        templates=self.templates)
            
                else :
                    pattern_to_give = list()
                    if self.re_order :
                        if len(self.slots_to_ask) == 1 :
                            pattern_to_give.append("re_order")
                            
                        next_state = random.sample(self.slots_to_ask,1)[0]
                    else :
                        next_state = self.slots_to_ask[0]
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=[next_state],
                                        values=None,
                                        message="request for {}".format(next_state),
                                        slot_concerned=next_state,
                                        templates=self.templates,
                                        pattern_marker=pattern_to_give)        
        else :
            
            
            next_state = "change_account"
            slot_message = ",".join(user_action.get_slots())
            
            key = "change_account"
            
            #bot_message = "It seems that you have not entered a valid account, you available accounts are {}, would you like change the source account ?".format(slot_message)
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_ACCOUNT",
                                slot_concerned="user_account",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def change_account_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_description() == "ACCEPT_REQUEST" :
            
            self.slots_to_ask.insert(0,"user_account")
            
            
            if self.audit_more :
                next_state = "check_account"
                
                key = "check_account"
                
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(self.user_values["user_account"])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["user_account"],
                                    values=None,
                                    #message="api_call:account_check_api, user_account:{}".format(self.user_values["user_account"]),
                                    message=bot_message,
                                    description="API_ACCOUNT_CHECK",
                                    slot_concerned="user_account",
                                    templates=self.templates)
                
                slot_changed = user_action.get_slots()[0]
                appropriate_state = self.states[slot_changed]
                priority_state, priority_action = appropriate_state(user_action)
                
                self.priority_states.append(priority_state)
                self.priority_actions[priority_state] = priority_action
                
                
                
            elif self.turn_compression :
                next_state = "check_account"
                
                key = "check_account"
                
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(self.user_values["user_account"])
                # perform the corresponding bot information
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["user_account"],
                                    values=None,
                                    #message="api_call:account_check_api, user_account:{}".format(self.user_values["user_account"]),
                                    message=bot_message,
                                    description="API_ACCOUNT_CHECK",
                                    slot_concerned="user_account",
                                    templates=self.templates)
            else :
                next_state = "user_account"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="Requesting user to provide new user account",
                                    slot_concerned="user_account",
                                    templates=self.templates)
        
        else :
            
            next_state = "end_call"
            
            key = "end_call_denied_change_account"
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="You denied to change the account",
                                message=bot_message,
                                slot_concerned="user_account",
                                templates=self.templates)
        
        return next_state , bot_action 
    
    def partner_state(self,user_action) :
        # remove the slot already asked
        self.remove_informed_slots(user_action)
            
        # update the user information with the new values got
        self.record_user_values(user_action)
        
        if "partner_name" in user_action.get_slots() :
            
            # sample out a new state based on the remaining slots to ask
            next_state = "check_partner"
            
            key = "check_partner"
            
            bot_message = self.templates[key][0]
            bot_message = bot_message.format(self.user_values["partner_name"])

            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["partner_name"],
                                values=None,
                                #message="api_call:partner_check_api, partner_name:{}".format(self.user_values["partner_name"]),
                                message=bot_message,
                                description="API_PARTNER_NAME_CHECK",
                                slot_concerned="partner_name",
                                templates=self.templates)
        
        else :
            
            if user_action.get_description() == "ANOTHER_SLOT_VALUE" :
                
                slot_given = user_action.get_slots()[0]
                appropriate_state = self.states[slot_given]
                
                next_state, bot_action = appropriate_state(user_action)
            
            else :
                
                next_state = "partner_name"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=["partner_name"],
                                    values=None,
                                    message="provide the Name of the Receiver",
                                    slot_concerned="partner_name",
                                    templates=self.templates)
        
        return next_state , bot_action
    
    def check_partner_name_state(self,user_action) :
        
        if "api_result:success" in user_action.get_message() :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
                
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                
                key = "confirm_transaction"
                
                bot_message = self.templates[key][0]
                
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    #message="confirm transaction ?",
                                    message=bot_message,
                                    description="CONFIRM_TRANSACTION",
                                    slot_concerned="api",
                                    templates=self.templates)
                
            else :
                pattern_to_give = list()
                if self.re_order :
                    if len(self.slots_to_ask) == 1 :
                        pattern_to_give.append("re_order")
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="Requesting the name for {}".format(next_state),
                                    slot_concerned=next_state,
                                    templates=self.templates,
                                    pattern_marker=pattern_to_give)
        
        else :
            
            next_state = "change_partner_name"
            slot_message = ",".join(user_action.get_slots())
            
            key = "change_partner_name"
            bot_message = self.templates[key][0]
            bot_message = bot_message.format(slot_message)
            #bot_message = "The recipient you are trying to provide doesn't exist, available list of recipients is {}, would you like to change the recipient ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_PARTNER_NAME",
                                slot_concerned="partner_name",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def change_partner_name_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_description() == "ACCEPT_REQUEST" :
            
            self.slots_to_ask.insert(0,"partner_name")
            if self.audit_more :
                next_state = "check_partner"
                
                key = "check_partner"
                
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(self.user_values["partner_name"])
                
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["partner_name"],
                                    values=None,
                                    #message="api_call:partner_check_api, partner_name:{}".format(self.user_values["partner_name"]),
                                    message=bot_message,
                                    description="API_PARTNER_NAME_CHECK",
                                    slot_concerned="partner_name",
                                    templates=self.templates)
                
                slot_changed = user_action.get_slots()[0]
                appropriate_state = self.states[slot_changed]
                priority_state, priority_action = appropriate_state(user_action)
                
                self.priority_states.append(priority_state)
                self.priority_actions[priority_state] = priority_action
                
            elif self.turn_compression :
                # sample out a new state based on the remaining slots to ask
                next_state = "check_partner"
                
                key = "change_partner"
                bot_message = self.templates[key]
                bot_message = bot_message.format(self.user_values["partner_name"])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["partner_name"],
                                    values=None,
                                    #message="api_call:partner_check_api, partner_name:{}".format(self.user_values["partner_name"]),
                                    message=bot_message,
                                    description="API_PARTNER_NAME_CHECK",
                                    slot_concerned="partner_name",
                                    templates=self.templates)
            else :
                
                next_state = "partner_name"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="Requesting user to provide new user account",
                                    slot_concerned="partner_name",
                                    templates=self.templates)
        
        else :
            
            next_state = "end_call"
            
            key = "end_call_denied_change_partner_name"
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="User failed to change partner name",
                                message=bot_message,
                                slot_concerned="partner_name",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "amount" in user_action.get_slots() :
             
            if "user_account" in self.user_values.keys() :
                
                # No random check this time because we have to check if the amount given is correct or not
                next_state = "check_amount"
                
                key = "check_amount"
                
                bot_message = self.templates[key][0]
                bot_message = bot_message.format(self.user_values["user_account"],self.user_values["amount"])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["limit","balance"],
                                    values=None,
                                    #message="api_call:check_amount, user_account:{}, amount:{}".format(self.user_values["user_account"],self.user_values["amount"]),
                                    message=bot_message,
                                    description="API_AMOUNT_CHECK",
                                    slot_concerned="amount",
                                    templates=self.templates)
            
            else :
                pattern_to_give = list()
                if self.re_order :
                    if len(self.slots_to_ask) ==  1 :
                        pattern_to_give.append("re_order")
                        
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting user to provide {} ".format(next_state),
                                    slot_concerned=next_state,
                                    templates=self.templates)
        
        else :
            
            if user_action.get_description() == "ANOTHER_SLOT_VALUE" :
                
                slot_given = user_action.get_slots()[0]
                appropriate_state = self.states[slot_given]
                
                next_state, bot_action = appropriate_state(user_action)
            else :
                
                next_state = "amount"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=["amount"],
                                    values=None,
                                    message="requesting the user to provide the Amount",
                                    slot_concerned="amount",
                                    templates=self.templates)
            
        return next_state , bot_action

    
    def check_amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        
        if "api_result:success" in user_action.get_message() :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                
                key = "confirm_transaction"
                
                bot_message = self.templates[key][0]
                
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    #message="confirm transaction ?",
                                    message=bot_message,
                                    description="CONFIRM_TRANSACTION",
                                    slot_concerned="api",
                                    templates=self.templates)
            
            else :
                pattern_to_give = list()
                if self.re_order :
                    if len(self.slots_to_ask) == 1:
                        pattern_to_give.append("re_order")
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    slot_concerned=next_state,
                                    templates=self.templates,
                                    pattern_marker=pattern_to_give)

        else :
            
            next_state = "change_amount"
            
            key = "change_amount"
            bot_message = self.templates[key][0]
            bot_message = bot_message.format(self.user_values["limit"],self.user_values["balance"],self.user_values["max_transferable_amt"])
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                #message="It seems the amount you provided can't be processed because your transaction limit is {} and your current balance is {} so the maximum you can transfer is {}, would you like to reduce your amount to this amount ?".format(self.user_values["limit"],self.user_values["balance"],self.user_values["max_transferable_amt"]),
                                message=bot_message,
                                description="CHANGE_TO_MAX_TRANSFERABLE_AMT",
                                slot_concerned="amount",
                                templates=self.templates)
        
        return next_state , bot_action
    
    
    def change_amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        if user_action.get_description() == "ACCEPT_REQUEST" :
            
            #self.slots_to_ask.insert(0,"amount")
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                
                key = "confirm_transaction"
                
                bot_message = self.templates[key][0]
                
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    #message="confirm transaction ?",
                                    message=bot_message,
                                    slot_concerned="api",
                                    templates=self.templates)
            
            else :
                
                pattern_to_give = list()
                if self.re_order :
                    if len(self.slots_to_ask) == 1:
                        pattern_to_give.append("re_order")
                    next_state = random.sample(self.slots_to_ask,1)[0]
                else :
                    next_state = self.slots_to_ask[0]
                    
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    slot_concerned=next_state,
                                    templates=self.templates,
                                    pattern_marker=pattern_to_give)
        
        else :
            
            next_state = "end_call"
            
            key = "end_call_denied_change_amount"
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="Rejected to change the decision",
                                message=bot_message,
                                slot_concerned="api",
                                templates=self.templates)
        
        return next_state , bot_action
    
    # This is a dead state and I don't know why it is here , but I don't know what will happen if I remove this function
    def balance_state(self,user_action) :
        return
    
    # ask for confirmation
    def confirmation_state(self,user_action) :
        
        if user_action.get_description() == "ACCEPT_REQUEST" :
            
            next_state = "api_call"
            #api_value = self.user_values["user_account"] + " " + self.user_values["partner_name"] + " "  + str(self.user_values["amount"])
            
            key = "transaction_call"
            bot_message = self.templates[key][0]
            bot_message = bot_message.format(self.user_values["user_account"],self.user_values["partner_name"],self.user_values["amount"])
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["user_account","partner_name","amount"],
                                values={"user_account" : self.user_values["user_account"],
                                        "partner_name" : self.user_values["partner_name"],
                                        "amount" : self.user_values["amount"]},
                                #message="api_call:transaction_api, user_account:{}, partner_name:{}, amount:{}".format(self.user_values["user_account"],self.user_values["partner_name"],self.user_values["amount"]),
                                message=bot_message,
                                description="API_CALL",
                                slot_concerned="api",
                                templates=self.templates)
        
        else :
            
            next_state = "end_call"
            
            key = "end_call_denied_confirm_transaction"
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="User refused to confirm transaction",
                                message=bot_message,
                                slot_concerned="api",
                                templates=self.templates)
        
        return next_state, bot_action

    # end the call
    def end_call_state(self,user_action) :
        
        #print("inside end_call state")
        next_state = "initial"
        bot_action = None
        
        return next_state , bot_action
    
    # Api call state
    def api_call_state(self,user_action) :
        
        if "api_result:success" in user_action.get_message() :
            
            key = "transaction_call_success"
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="{} conducted successfully, ciao !!".format(self.user_values["intent"]),
                                message=bot_message,
                                slot_concerned="api",
                                templates=self.templates)
        
        else :
            
            key = "transaction_call_failed"
            
            bot_message = self.templates[key][0]
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                #message="error in processing {}".format(self.user_values["intent"]),
                                message=bot_message,
                                slot_concerned="api",
                                templates=self.templates)
        
        next_state = "end_call"
        
        return next_state , bot_action

### The dialog generator (the simplest piece of the code)

Here we are generating dialogs based on "Actions".
We always keep track of the last transaction performed. If last_action is an *"end_call"* then we conclude our dialog.
We do this for each of the 50 dialogs

In [10]:
def create_dialogs(User,
                   Bot,
                   number_of_dialogs,
                   dialog_templates=None,
                   turn_compression=False,
                   new_api=False,
                   re_order=False,
                   another_slot=False,
                   audit_more=False) :
    
    dialogs_train = list()
    dialogs_val = list()
    dialogs_test = list()
    dialogs_test_oot = list()
    
    final_dialogs = dict()
    
    for i in range(number_of_dialogs) :
        
        dialog_train = list()
        dialog_val = list()
        dialog_test = list()
        dialog_test_oot = list()

        user = User(templates=dialog_templates["train"],
                    list_of_user_profiles=list_of_user_profiles,
                    user_values=user_values,
                    turn_compression=turn_compression,
                    new_api=new_api,
                    another_slot=another_slot,
                    audit_more=audit_more)
        
        bot = Bot(templates=dialog_templates["train"],
                  turn_compression=turn_compression,
                  re_order=re_order,
                  audit_more=audit_more)

        user_action_train = Action(actor="User",
                             action=None,
                             slots=None,
                             values=None,
                             message="<SILENCE>",
                             templates=dialog_templates["train"])
        

        bot_action_train = Action(actor="Bot",
                            action="request",
                            slots=["intent"],
                            values=None,
                            message="Gettinng intent",
                            templates=dialog_templates["train"])
        
        
        
        dialog_train.append(user_action_train)
        dialog_train.append(bot_action_train)
        
        # creating validation actions
        user_action_val = copy.deepcopy(user_action_train)
        bot_action_val = copy.deepcopy(bot_action_train)
        
        
        
        user_action_val.set_templates(new_templates=dialog_templates["val"])
        bot_action_val.set_templates(new_templates=dialog_templates["val"])
        
        
        dialog_val.append(user_action_val)
        dialog_val.append(bot_action_val)
        
        # creating test actions
        user_action_test = copy.deepcopy(user_action_train)
        bot_action_test = copy.deepcopy(bot_action_train)
        
        
        user_action_test.set_templates(new_templates=dialog_templates["test"])
        bot_action_test.set_templates(new_templates=dialog_templates["test"])
        
        dialog_test.append(user_action_test)
        dialog_test.append(bot_action_test)
        
        # creating test oot actions
        user_action_test_oot = copy.deepcopy(user_action_train)
        bot_action_test_oot = copy.deepcopy(bot_action_train)
        
        
        user_action_test_oot.set_templates(new_templates=dialog_templates["test_oot"])
        bot_action_test_oot.set_templates(new_templates=dialog_templates["test_oot"])
        
        dialog_test_oot.append(user_action_test_oot)
        dialog_test_oot.append(bot_action_test_oot)
        
        
        latest_action = None
        
        while latest_action != "end_call" :
            
            user_action_train = user.speak(bot_action_train)
            
            #print("user_action {}, user_message {} user_description {}".format(user_action_train.get_action(),user_action_train.get_message(),user_action_train.get_description()))
            bot_action_train = bot.speak(user_action_train)
            #print("bot_action {}, bot_message {} bot_description {}".format(bot_action_train.get_action(),bot_action_train.get_message(),bot_action_train.get_description()))
            latest_action = bot_action_train.get_action()
            
            # creating validation actions
            user_action_val = copy.deepcopy(user_action_train)
            bot_action_val = copy.deepcopy(bot_action_train)
            
            
            # creating test actions
            user_action_test = copy.deepcopy(user_action_train)
            bot_action_test = copy.deepcopy(bot_action_train)
            
            # creating test oot actions
            user_action_test_oot = copy.deepcopy(user_action_train)
            bot_action_test_oot = copy.deepcopy(bot_action_train)
            
            # setting validation templates
            user_action_val.set_templates(new_templates=dialog_templates["val"])
            bot_action_val.set_templates(new_templates=dialog_templates["val"])
            
            # setting test templates
            user_action_test.set_templates(new_templates=dialog_templates["test"])
            bot_action_test.set_templates(new_templates=dialog_templates["test"])
            
            # setting test oot templates
            user_action_test_oot.set_templates(new_templates=dialog_templates["test_oot"])
            bot_action_test_oot.set_templates(new_templates=dialog_templates["test_oot"])
            
            dialog_train.append(user_action_train)
            dialog_train.append(bot_action_train)
            
            dialog_val.append(user_action_val)
            dialog_val.append(bot_action_val)
            
            dialog_test.append(user_action_test)
            dialog_test.append(bot_action_test)
            
            dialog_test_oot.append(user_action_test_oot)
            dialog_test_oot.append(bot_action_test_oot)
            
            #print("User:{} Bot:{}".format(user_action.get_message(),bot_action.get_message()))
        
        dialogs_train.append(dialog_train)
        dialogs_val.append(dialog_val)
        dialogs_test.append(dialog_test)
        dialogs_test_oot.append(dialog_test_oot)
        
    final_dialogs["train"] = dialogs_train
    final_dialogs["val"] = dialogs_val
    final_dialogs["test"] = dialogs_test
    final_dialogs["test_oot"] = dialogs_test_oot
    
    return final_dialogs

In [11]:
required_number_of_dialogs = 200

In [12]:
transaction_dialogs = create_dialogs(User=Transaction_user,
                                     Bot=Transaction_bot,
                                     number_of_dialogs=required_number_of_dialogs,
                                     dialog_templates=transaction_templates,
                                     turn_compression=False,
                                     new_api=False,
                                     re_order=False,
                                     another_slot=False,
                                     audit_more=False)

print("length of training transaction dialogs :{}".format(str(len(transaction_dialogs["train"]))))
print("length of validation transaction dialogs :{}".format(str(len(transaction_dialogs["val"]))))
print("length of test transaction dialogs : {}".format(str(len(transaction_dialogs["test_oot"]))))

'''transaction_dialogs_turn_compression = create_dialogs(User=Transaction_user,
                                                      Bot=Transaction_bot,
                                                      number_of_dialogs=required_number_of_dialogs,
                                                      dialog_templates=transaction_templates,
                                                      turn_compression=True,
                                                      new_api=False,
                                                      re_order=False,
                                                      another_slot=False,
                                                      audit_more=False)

print("length of training transaction dialogs for turn compression test :{}".format(str(len(transaction_dialogs_turn_compression["test"]))))
print("length of test transaction dialogs for turn compressioon test with oot : {}".format(str(len(transaction_dialogs_turn_compression["test_oot"]))))

transaction_dialogs_new_api = create_dialogs(User=Transaction_user,
                                             Bot=Transaction_bot,
                                             number_of_dialogs=required_number_of_dialogs,
                                             dialog_templates=transaction_templates,
                                             turn_compression=False,
                                             new_api=True,
                                             re_order=False,
                                             another_slot=False,
                                             audit_more=False)

print("length of training transaction dialogs for new api test :{}".format(str(len(transaction_dialogs_new_api["test"]))))
print("length of test transaction dialogs for new api test with oot : {}".format(str(len(transaction_dialogs_new_api["test_oot"]))))

transaction_dialogs_re_order = create_dialogs(User=Transaction_user,
                                              Bot=Transaction_bot,
                                              number_of_dialogs=required_number_of_dialogs,
                                              dialog_templates=transaction_templates,
                                              turn_compression=False,
                                              new_api=False,
                                              re_order=True,
                                              another_slot=False,
                                              audit_more=False)

print("length of training transaction dialogs for re-order test :{}".format(str(len(transaction_dialogs_re_order["test"]))))
print("length of test transaction dialogs for re-order test with oot : {}".format(str(len(transaction_dialogs_re_order["test_oot"]))))

transaction_dialogs_another_slot = create_dialogs(User=Transaction_user,
                                                  Bot=Transaction_bot,
                                                  number_of_dialogs=required_number_of_dialogs,
                                                  dialog_templates=transaction_templates,
                                                  turn_compression=False,
                                                  new_api=False,
                                                  re_order=False,
                                                  another_slot=True,
                                                  audit_more=False)


print("length of training transaction dialogs for another slot test : {}".format(str(len(transaction_dialogs_another_slot["test"]))))
print("lenght of test transaction dialogs for another slot with oot : {}".format(str(len(transaction_dialogs_another_slot["test_oot"]))))

transaction_dialogs_audit_more = create_dialogs(User=Transaction_user,
                                                Bot=Transaction_bot,
                                                number_of_dialogs=required_number_of_dialogs,
                                                dialog_templates=transaction_templates,
                                                turn_compression=False,
                                                new_api=False,
                                                re_order=False,
                                                another_slot=False,
                                                audit_more=True)

print("length of transaction dialogs test : {}".format(str(len(transaction_dialogs_audit_more["test"]))))
print("length of transaction dialogs test oot : {}".format(str(len(transaction_dialogs_audit_more["test_oot"]))))'''

length of training transaction dialogs :200
length of validation transaction dialogs :200
length of test transaction dialogs : 200


'transaction_dialogs_turn_compression = create_dialogs(User=Transaction_user,\n                                                      Bot=Transaction_bot,\n                                                      number_of_dialogs=required_number_of_dialogs,\n                                                      dialog_templates=transaction_templates,\n                                                      turn_compression=True,\n                                                      new_api=False,\n                                                      re_order=False,\n                                                      another_slot=False,\n                                                      audit_more=False)\n\nprint("length of training transaction dialogs for turn compression test :{}".format(str(len(transaction_dialogs_turn_compression["test"]))))\nprint("length of test transaction dialogs for turn compressioon test with oot : {}".format(str(len(transaction_dialogs_turn_compression["t

### Printing the dialogs

We have to remeber that the dialog is a set of action and so to format them into a useful information here is the protocol that we will follow :
1. if its a request or inform action then we check if there is a slot associated.
    
    a. if there is then we know that the action is a request/inform with a slot and value associated.
    
    b. If there is no slot then we are assuming it's the case when there is an unexpected action occuring and in this case we print the message associated with the action.
    
2. If it's an api call then print the api_call with the appropriate values

3. If it's an end_call then print the message that is given with the ending of the call.

### How to Read the Data :

Every dialog is a set of actions.

When printing it is printed as :

{actor} : {action} {slots_asked} ==> {slots_given(if any)}

So the Bot actions are printed as follows 

Bot : { request/api_call/end_call }  {[slots asked]/message given(if any)}

And the User actions are printed as follows

User : {inform} {[slots given]} {[slot values]/message}

On Similar lines, the Transaction_Check_Software and Balance_check_Software follow the format of Bot action and Software follows the format of User action

### How to Write the Data to File

There are three files that are used for the puprose here :
a. raw_data.txt
b. train_data.txt
c. candidate.txt

**a. raw_data.txt**

This file is supposed to print the data in a human-readable format. The General representation of a dialog is as folows :
    
    1. {Actor} : {Dialog}
    so a good example of this format is -:
        1. User : <SILENCE>
        2. Bot : How can I help you today
        3. User : I would like to know my account balance
        4. API : request list_of_accounts:{name=Sourabh}
        5. API_RESP : inform list_of_accounts:{Credit,Savings}
        6. Bot : You have the following list of accounts : Credit,Savings , which one ?
        7. User : Let's see Savings
        8. Bot : api_call account:{Savings}
        9. API_RESP : inform account:{Savings} balance:{xxx}
        10.Bot : Your balance for Savings account is xxx euros, ciao !!
      
**b. train_data.txt**

The file is supposed to be written as the file for the training data in the conversation.
A Sample Conversation is written below :

    1 <SILENCE>	how can i help you today ?
    2 I would like to see my transaction history	request_accounts accounts:Simone
    3 inform list_of_accounts : Savings	You have the following accounts : Savings , which one do you wish ?
    4 Savings	Which information should I give credit or debit ?
    5 let's see debit	Can I ask what is the name of the partner ?
    6 it's for Serra	api_call Savings debit Serra
    7 inform api_call : success	end_call api_call successful !!
    
**c. candidate.txt**

The file is suppose to create the candidates for the appropriate conversations.
A Sample of candidates is written below.

    1 amount_check  user_account:Savings amount:9000
    1 It seems the amount you provided can't be processed because your transaction limit is 5000 and your current balance is 8000 so the maximum you can transfer is 5000, would you like to reduce your amount to this amount ?
    1 The recipient you are trying to provide doesn't exist, available list of recipients is Tahir,Sourabh,Samuel,Vevake, would you like to change the recipient ?
    1 It seems that you have not entered a valid account, you available accounts are Savings, would you like change the source account ?
    1 api_call Savings Marco 400
    1 The recipient you are trying to provide doesn't exist, available list of recipients is Sourabh,Matteo, would you like to change the recipient ?
    1 api_call Credit Samuel 400
    1 destination_name_check destination_name:Vevake
    1 amount_check user_account:Checkin , amount:1600
    1 account_check user_account:Credit
    1 amount_check  user_account:Credit amount:2400
    1 api_call Savings Marco 4500

**Writing Raw Data for the Transaction Domain**

The code below writes the training data for the Transaction Domain Memory Network in a human readable format such that one can asses the quality of the conversation.

In [13]:
transaction_dialogs_all = list()
transaction_dialogs_all.extend(transaction_dialogs["train"])
transaction_dialogs_all.extend(transaction_dialogs["val"])

transaction_dialogs_all.extend(transaction_dialogs["test"])
transaction_dialogs_all.extend(transaction_dialogs["test_oot"])

'''transaction_dialogs_all.extend(transaction_dialogs_turn_compression["test"])
transaction_dialogs_all.extend(transaction_dialogs_turn_compression["test_oot"])

transaction_dialogs_all.extend(transaction_dialogs_new_api["test"])
transaction_dialogs_all.extend(transaction_dialogs_new_api["test_oot"])

transaction_dialogs_all.extend(transaction_dialogs_re_order["test"])
transaction_dialogs_all.extend(transaction_dialogs_re_order["test_oot"])

transaction_dialogs_all.extend(transaction_dialogs_another_slot["test"])
transaction_dialogs_all.extend(transaction_dialogs_another_slot["test_oot"])

transaction_dialogs_all.extend(transaction_dialogs_audit_more["test"])
transaction_dialogs_all.extend(transaction_dialogs_audit_more["test_oot"])
'''
random.shuffle(transaction_dialogs_all)

**Create Raw Data**

The function below converts the given set of dialogs into a human readable file and writes it to the appropriate file

In [14]:
def create_raw_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
    
    file_handle = open(os.path.join(file_directory,file_name),"w")
    
    for dialog in dialogs :
        if dialog :
            for action in dialog :
                if action :
                    file_handle.write(action.get_dialog())
                    file_handle.write("\n")
            file_handle.write("\n")
    file_handle.close()

**Writing Raw Data for the Transaction Domain**

The code below writes the training data for the Transaction Domain Memory Network in a human readable format such that one can asses the quality of the conversation.

In [15]:
create_raw_data(file_directory="../data/transaction_data/",file_name="raw_data_train.txt",dialogs=transaction_dialogs["train"])
create_raw_data(file_directory="../data/transaction_data/",file_name="raw_data_val.txt",dialogs=transaction_dialogs["val"])

In [16]:
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_test.txt",dialogs=transaction_dialogs["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_test_oot.txt",dialogs=transaction_dialogs["test_oot"])

'''create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_turn_compression_test.txt",dialogs=transaction_dialogs_turn_compression["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_turn_compression_test_oot.txt",dialogs=transaction_dialogs_turn_compression["test_oot"])

create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_new_api_test.txt",dialogs=transaction_dialogs_new_api["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_new_api_test_oot.txt",dialogs=transaction_dialogs_new_api["test_oot"])

create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_re_order_test.txt",dialogs=transaction_dialogs_re_order["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_re_order_test_oot.txt",dialogs=transaction_dialogs_re_order["test_oot"])

create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_another_slot_test.txt",dialogs=transaction_dialogs_another_slot["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_another_slot_test_oot.txt",dialogs=transaction_dialogs_another_slot["test_oot"])

create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_audit_more_test.txt",dialogs=transaction_dialogs_audit_more["test"])
create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_audit_more_test_oot.txt",dialogs=transaction_dialogs_audit_more["test_oot"])'''

'create_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_turn_compression_test.txt",dialogs=transaction_dialogs_turn_compression["test"])\ncreate_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_turn_compression_test_oot.txt",dialogs=transaction_dialogs_turn_compression["test_oot"])\n\ncreate_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_new_api_test.txt",dialogs=transaction_dialogs_new_api["test"])\ncreate_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_new_api_test_oot.txt",dialogs=transaction_dialogs_new_api["test_oot"])\n\ncreate_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_re_order_test.txt",dialogs=transaction_dialogs_re_order["test"])\ncreate_raw_data(file_directory="../data/transaction_data/test/",file_name="raw_data_re_order_test_oot.txt",dialogs=transaction_dialogs_re_order["test_oot"])\n\ncreate_raw_data(file_directory="../data/t

**Create Training Data Function**

The code below creates the training data from the provided dialogs and writes to the appropriate file

In [17]:
def create_training_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for dialog in dialogs :
        count = 1
        for i in range(0,len(dialog),2) :
            user_dialog = dialog[i]
            bot_dialog = dialog[i+1]
            
            user_pattern = user_dialog.get_pattern_marker()
            bot_pattern = bot_dialog.get_pattern_marker()
            
            if user_pattern or bot_pattern :
                list_of_pattern = list()
                if user_pattern :
                    list_of_pattern.extend(user_pattern)
                if bot_pattern :
                    list_of_pattern.extend(bot_pattern)
                    
                pattern_marked = '-'.join(list_of_pattern)
                file_handle.write("{}{} {}\t{}\n".format(str(count),pattern_marked,user_dialog.get_dialog(with_actor=False),bot_dialog.get_dialog(with_actor=False)))
            else :
                file_handle.write("{} {}\t{}\n".format(str(count),user_dialog.get_dialog(with_actor=False),bot_dialog.get_dialog(with_actor=False)))
            count += 1
        file_handle.write("\n")
    file_handle.close()

**Training Data for Transaction Domain**

The code below creates the training data for the Transaction Memory Network(The Memory Network responsible for handling the Transaction Intent).

In [18]:
create_training_data(file_directory="../data/transaction_data/",file_name="train_data.txt",dialogs=transaction_dialogs["train"])
create_training_data(file_directory="../data/transaction_data/",file_name="val_data.txt",dialogs=transaction_dialogs["val"])

In [19]:
create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data.txt",dialogs=transaction_dialogs["test"])
create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_oot.txt",dialogs=transaction_dialogs["test_oot"])

'''create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_turn_compression_test.txt",dialogs=transaction_dialogs_turn_compression["test"])
create_training_data(file_directory="../data/transaction_data/test",file_name="test_data_turn_compression_test_oot.txt",dialogs=transaction_dialogs_turn_compression["test_oot"])

create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_new_api_test.txt",dialogs=transaction_dialogs_new_api["test"])
create_training_data(file_directory="../data/transaction_data/test",file_name="test_data_new_api_test_oot.txt",dialogs=transaction_dialogs_new_api["test_oot"])

create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_re_order_test.txt",dialogs=transaction_dialogs_re_order["test"])
create_training_data(file_directory="../data/transaction_data/test",file_name="test_data_re_order_test_oot.txt",dialogs=transaction_dialogs_re_order["test_oot"])

create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_another_slot_test.txt",dialogs=transaction_dialogs_another_slot["test"])
create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_another_slot_test_oot.txt",dialogs=transaction_dialogs_another_slot["test_oot"])

create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_audit_more_test.txt",dialogs=transaction_dialogs_audit_more["test"])
create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_audit_more_test_oot.txt",dialogs=transaction_dialogs_audit_more["test_oot"])'''

'create_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_turn_compression_test.txt",dialogs=transaction_dialogs_turn_compression["test"])\ncreate_training_data(file_directory="../data/transaction_data/test",file_name="test_data_turn_compression_test_oot.txt",dialogs=transaction_dialogs_turn_compression["test_oot"])\n\ncreate_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_new_api_test.txt",dialogs=transaction_dialogs_new_api["test"])\ncreate_training_data(file_directory="../data/transaction_data/test",file_name="test_data_new_api_test_oot.txt",dialogs=transaction_dialogs_new_api["test_oot"])\n\ncreate_training_data(file_directory="../data/transaction_data/test/",file_name="test_data_re_order_test.txt",dialogs=transaction_dialogs_re_order["test"])\ncreate_training_data(file_directory="../data/transaction_data/test",file_name="test_data_re_order_test_oot.txt",dialogs=transaction_dialogs_re_order["test_oot"])\n\ncreate_t

**Create Candidate Function**

The function below creates the candidate list from the provided dialogs and writes it to the appropriate file

In [20]:
def create_candidates(file_directory="../data/",file_name="data.txt",dialogs=None) :
    candidate_set = set()
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    for dialog in dialogs :
        for action in dialog :
            if action.get_actor() == "Bot" :
                candidate_set.add(action.get_dialog(with_actor=False))
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for candidate in candidate_set :
        file_handle.write("1 {}\n".format(candidate))
    file_handle.close()

**Writing Transaction Dialog Candidates**

The code below creates the candidate file for the conversations in the Transaction Domain, i.e Bot Utterances in the Transaction Domain

In [21]:
create_candidates(file_directory="../data/transaction_data/",file_name="candidates.txt",dialogs=transaction_dialogs_all)

In [22]:
def find_generic_responses(actor=None,dialogs=None,file_directory=None,file_name=None) :
    set_of_sentences = set()
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
    
    for dialog in dialogs :
        for action in dialog :
            if action.get_actor() == actor and action.template_found() == False :
                 if "api_call" not in action.get_dialog(with_actor=False) :
                        set_of_sentences.add(action.get_dialog(with_actor=False))
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for sentence in set_of_sentences :
        file_handle.write("1 {}\n".format(sentence))
    file_handle.close()

In [23]:
find_generic_responses(actor="User",dialogs=transaction_dialogs_all,file_directory="../data/transaction_data/",file_name="user_generic_responses.txt")
find_generic_responses(actor="Bot",dialogs=transaction_dialogs_all,file_directory="../data/transaction_data/",file_name="bot_generic_responses.txt")